# 00. Capture and Mimic First Run

Procedure:

Settings > Apps > Meijer.

1. Force Stop.
2. Clear Data.

Start from complete scratch for mitmproxy.

1. Launch mitmproxy, if not already running.
2. Launch Meijer app.

There is a lot of 'chatter', requests blocked by pihole don't need replicated.

Right now just working on replicating requests, not comprehending them.

In [1]:
import requests

#### Request 1

In [2]:
request=dict()
request["url"] = "https://meijer.122.2o7.net/id"
request["headers"] = {
    'user-agent': 'Dalvik/2.1.0 (Linux; U; Android 5.0.2; HTCONE Build/LRX22G)'
}
r = requests.get(**request)
assert r.status_code==200
r.json() # An ID used for?

{'id': '2E0A2ACE05079F09-4000011C6000CD25'}

#### Request 2

In [3]:
request=dict()
request["url"] = "https://static.meijer.com/mobileassets/info/mma_config.json"
request["headers"] = {
    'Platform': 'Android',
    'Version': '5.10.0',
    'Build': '51000000',
    'User-Agent': 'okhttp/3.8.0'
}
r = requests.get(**request)
assert r.status_code==200
r.json() # Determine if the user needs to update.

{'forceUserAccountsToUpdate': False,
 'enableAisleFire': False,
 'currentAppVersion': {'major': 5, 'minor': 9, 'patch': 0},
 'currentAppVersionIOS': {'major': 5, 'minor': 9, 'patch': 0},
 'currentAppVersionAndroid': {'major': 5, 'minor': 9, 'patch': 0},
 'forceAppVersionIOS': {'major': 4, 'minor': 3, 'patch': 0, 'minOS': 9},
 'forceAppVersionAndroid': {'major': 4, 'minor': 3, 'patch': 0, 'minSdk': 19},
 'updateAppMessageTitle': 'New version available',
 'updateAppMessage': 'Update now to get the latest features.',
 'promptToAddmPerksCardInCreateAccountFlow': False,
 'enableRatingsPrompt': True,
 'percentChanceOfLoggingRatingsPromptEvent': 3,
 'ratingsPromptCriteria': {'clippedCouponsCount': 11}}

#### Request 3

The ```Authorization``` header is a base64 encoded string generated from:

```./res/values/strings.xml:    <string name="account_services_secret">drAqas76Re7RekeBanaMaNEMah7paDE5</string>```

In [4]:
import base64
# Working Backwards.
AUTH="bW1hOmRyQXFhczc2UmU3UmVrZUJhbmFNYU5FTWFoN3BhREU1"
base64.decodebytes(AUTH.encode("UTF-8"))

b'mma:drAqas76Re7RekeBanaMaNEMah7paDE5'

In [5]:
# Working Forward.
account_services_secret="drAqas76Re7RekeBanaMaNEMah7paDE5"
AUTH_=base64.encodebytes("mma:{}".format(account_services_secret).encode("UTF-8")).decode("UTF-8").strip()

In [6]:
assert AUTH==AUTH_

In [7]:
request=dict()
request["url"] = "https://login.meijer.com/as/token.oauth2"
request["headers"] = {
    'Authorization': 'Basic {}'.format(AUTH),
    'Platform': 'Android',
    'Version': '5.10.0',
    'Build': '51000000',
    'User-Agent': 'okhttp/3.8.0'
}
request["params"] =  {
    'grant_type': 'client_credentials',
    'scope': 'openid',
}
r = requests.post(**request)
assert r.status_code==200
r.json().keys() # Determine if the user needs to update.

dict_keys(['access_token', 'token_type', 'expires_in'])

In [8]:
access_token = r.json()["access_token"] # We'll need this later.

#### Request 4

Get ads and dynamic content for the Meijer App.

Relevant source file: ```./smali/com/meijer/mobile/serverapi/rxjava/DigitalMperksMMAAPIObservable.smali`` 

In [9]:
request=dict()
request["url"] = "https://mperksservices.meijer.com/dgtlmPerksMMA/api/cms/home/content"
request["headers"] = {
    'Accept': 'application/vnd.meijer.digitalmperks.homecontent-v1.0+json',
    'Authorization': 'Bearer {}'.format(access_token),
    'Content-Type': 'application/vnd.meijer.digitalmperks.homecontent-v1.0+json',
    'Platform': 'Android',
    'Version': '5.10.0',
    'Build': '51000000',
    'User-Agent': 'okhttp/3.8.0'
}
r = requests.get(**request)
r

<Response [200]>

In [11]:
r.json()

{'bottomBanners': [{'deeplinkURL': 'meijerapp://mperks/bundle?tagName=2018dailydeals',
   'bannerURL': 'http://static.meijer.com/mktg/web/mPerks/weekly/2018/1130/2018_DailyDeals_Homepage_App.jpg',
   'locationIndex': 1.0,
   'analyticsTag': '2018dailydeals'}],
 'topBanner': {'deeplinkURL': 'meijerapp://mperks/bundle?tagName=20181213mperksevent',
  'bannerURL': 'http://static.meijer.com/mktg/web/mPerks/weekly/2018/1209/20181213_Hook_App.jpg',
  'locationIndex': 0.0,
  'analyticsTag': 'Corporate_Hook'},
 'specialOffersBannerURL': 'http://static.meijer.com/mktg/web/mPerks/weekly/HPO_Banners/2018/12_December/2018_December_App_1024x262.jpg'}

In [12]:
import IPython.display

In [13]:
IPython.display.Image(url=r.json()["specialOffersBannerURL"])

In [14]:
IPython.display.Image(url=r.json()["topBanner"]["bannerURL"])